In [ ]:
from PIL import Image, ImageDraw, ImageFont
from typing import Iterable
import pandas as pd
import numpy as np
import os

In [ ]:
import matplotlib.pyplot as plt
label_dict = {'0':'vest', '1':'helmet'}
def draw(img, box, label, a_box=None, a_label=None):
    xmin, ymin, xmax, ymax = box
    if a_label is not None:
      print("Actual label: ", label_dict[str(a_label)])

    plt.imshow(img)
    plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, edgecolor='red', linewidth=2))
    plt.text(xmin, ymin - 10, label_dict[str(label)], color='red', backgroundcolor='white')
    if a_box is not None:
      xmin, ymin, xmax, ymax = a_box
      plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, edgecolor='green', linewidth=2))

    plt.axis('off')
    plt.show()

In [ ]:
def data_generator(path, batch_size):
    X = []
    annotations = pd.read_csv(path + '/_annotations.csv')
    annotations = annotations.sample(frac=1)  # Shuffle dataframe rows

    file_list = annotations['filename'].tolist()

    for idx, filename in enumerate(file_list):
        try:
            img = Image.open(os.path.join(path, filename))
            X.append(np.array(img))

            if len(X) == batch_size:
                labels = annotations.iloc[idx - batch_size + 1:idx + 1][['class']].values
                boxes = annotations.iloc[idx - batch_size + 1:idx + 1][['xmin', 'ymin', 'xmax', 'ymax']].values/224
                yield np.array(X, dtype=np.int32), to_categorical(labels, num_classes=NUM_CLASSES, dtype='int32'), boxes
                X = []

        except IOError:
            print('error')

    if len(X) > 0:
        labels = annotations.iloc[-len(X):][['class']].values,
        boxes = annotations.iloc[-len(X):][['xmin', 'ymin', 'xmax', 'ymax']].values/224
        yield np.array(X, dtype=np.int32), to_categorical(labels, num_classes=NUM_CLASSES, dtype='int32'), boxes


In [ ]:
def IoU_metric(y_true, y_pred):
    """
    The function computes the IoU score for each pair of (ground truth bounding box, predicted bounding box)
    y_true is a tensor representing the ground truth bounding boxes for a given batch of samples (in format xmin, ymin, xmax, ymax)
    y_pred is a tensor representing the predicted bounding boxes for the same batch of samples (in format xmin, ymin, xmax, ymax)
    """
    x1 = K.maximum(y_true[:, 0], y_pred[:, 0])
    y1 = K.maximum(y_true[:, 1], y_pred[:, 1])
    x2 = K.minimum(y_true[:, 2], y_pred[:, 2])
    y2 = K.minimum(y_true[:, 3], y_pred[:, 3])

    intersection = K.maximum(0.0, x2 - x1) * K.maximum(0.0, y2 - y1)

    y_true_area = (y_true[:, 2] - y_true[:, 0]) * (y_true[:, 3] - y_true[:, 1])
    y_pred_area = (y_pred[:, 2] - y_pred[:, 0]) * (y_pred[:, 3] - y_pred[:, 1])
    union = y_true_area + y_pred_area - intersection

    iou = intersection / (union + K.epsilon())
    return iou

In [ ]:
 def visualize_results(name, history):
    acc = history['classifier_accuracy']
    val_acc = history['val_classifier_accuracy']
    acc_loss = history['classifier_loss']
    val_acc_loss = history['val_classifier_loss']

    iou = history['regressor_IoU_metric']
    val_iou = history['val_regressor_IoU_metric']
    iou_loss = history['regressor_loss']
    val_iou_loss = history['val_regressor_loss']

    loss = history['loss']
    val_loss = history['val_loss']
    best_loss_epoch = np.argmin(val_loss)
    best_loss = np.min(val_loss)


    epochs = range(len(acc))
    plt.figure(figsize=(14, 8))

    # Accuracy
    plt.subplot(2, 2, 1)
    plt.plot(epochs, acc, 'b-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'g-', label='Validation accuracy')
    plt.plot(best_loss_epoch, val_acc[best_loss_epoch], 'ro', label='Selected model')
    plt.title('Accuracy on '+name)
    plt.legend(loc='lower right')

    # IoU
    plt.subplot(2, 2, 2)
    plt.plot(epochs, iou, 'b-', label='Training IoU')
    plt.plot(epochs, val_iou, 'g-', label='Validation IoU')
    plt.plot(best_loss_epoch, val_iou[best_loss_epoch], 'ro', label='Selected model')
    plt.title('Intersection over Union on '+name)
    plt.legend(loc='lower right')


    plt.figure(figsize=(14, 8))

    # Classifier loss
    plt.subplot(2, 2, 1)
    plt.plot(epochs, acc_loss, 'b-', label='Training classifier loss')
    plt.plot(epochs, val_acc_loss, 'g-', label='Validation classifier loss')
    plt.plot(best_loss_epoch, val_acc_loss[best_loss_epoch], 'ro', label='Selected model')
    plt.title('Classifier loss on '+name)
    plt.legend(loc='upper right')

    # Regressor loss
    plt.subplot(2, 2, 2)
    plt.plot(epochs, iou_loss, 'b-', label='Training regressor loss')
    plt.plot(epochs, val_iou_loss, 'g-', label='Validation regressor loss')
    plt.plot(best_loss_epoch, iou_loss[best_loss_epoch], 'ro', label='Selected model')
    plt.title('Regressor loss on '+name)
    plt.legend(loc='upper right')

    plt.show()
    print(f"Lowest validation loss has been reached at epoch {best_loss_epoch:d} with:")
    print(f"Training accuracy of {acc[best_loss_epoch]:.3f} and Validation accuracy of {val_acc[best_loss_epoch]:.3f} ")
    print(f"Training IoU of {iou[best_loss_epoch]:.3f} and Validation IoU of {val_iou[best_loss_epoch]:.3f}\n")

    plt.clf()
    plt.figure()
    plt.plot(epochs, loss, 'b-', label='Training loss')
    plt.plot(epochs, val_loss, 'g-', label='Validation loss')
    plt.plot(best_loss_epoch, best_loss, 'ro', label='Selected model')
    plt.title('Total loss on '+name)
    plt.legend(loc='upper right')